In [1]:
from pathlib import Path

from langchain import LLMMathChain
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI

from trulens_eval import Tru, TruChain, FeedbackMode
from trulens_eval.instruments import Instrument
from trulens_eval.sessions import SessionManager
from trulens_eval.messages import OpenAiFunctionsMessagesExtractor

In [2]:
tru = Tru()

In [3]:
tru.start_dashboard(force=True)

Force stopping dashboard ...
Starting dashboard ...


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at http://192.168.178.30:8501 .


In [4]:
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613")

llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)

tools = [
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to answer questions about math"
    ),
    Tool(
        name="Nothing",
        func=lambda x: x,
        description="useful for for nothing"
    ),
]

agent = initialize_agent(tools, llm, agent=AgentType.OPENAI_FUNCTIONS, verbose=True)

In [5]:
Instrument.Default.CLASSES.add(agent.agent.__class__)
Instrument.Default.MODULES.add(agent.agent.__module__)
Instrument.Default.METHODS["plan"] = lambda o: isinstance(o, agent.agent.__class__)

In [6]:
app = TruChain(
    app=agent,
    app_id="OpenAiFunctions",
)

In [7]:
session_manager = SessionManager(
    app=app,
    messages_extractor=OpenAiFunctionsMessagesExtractor.from_tools(tools)
)

In [8]:
queries = [
    "what is the square root of 123",
    "how much is Euler's number divided by PI"
]

with session_manager.start_session() as session_runner:
    for query in queries:
        session_runner(inputs={"input": query})

session_runner.get_messages() # messages extracted from the last session



> Entering new AgentExecutor chain...

Invoking: `Calculator` with `sqrt(123)`




> Entering new LLMMathChain chain...
sqrt(123)```text
sqrt(123)
```
...numexpr.evaluate("sqrt(123)")...

Answer: 11.090536506409418
> Finished chain.
Answer: 11.090536506409418The square root of 123 is approximately 11.0905.

> Finished chain.


> Entering new AgentExecutor chain...

Invoking: `Calculator` with `e / pi`




> Entering new LLMMathChain chain...
e / pi```text
e / pi
```
...numexpr.evaluate("e / pi")...

Answer: 0.8652559794322651
> Finished chain.
Answer: 0.8652559794322651Euler's number divided by PI is approximately 0.8652559794322651.

> Finished chain.


,session_id,ts,call_idx,source,label,content,metadata_,record_id,content_type
0,session_hash_dc51233f49a100fd2351e2108507aeb6,2023-08-10 16:27:16.091494,0,user,Input,what is the square root of 123,{'agent_type': 'openai-functions'},record_hash_2ab2cd6637940f8777b4f641cc3ceb3e,text
1,session_hash_dc51233f49a100fd2351e2108507aeb6,2023-08-10 16:27:17.400077,0,assistant,Action[Calculator],sqrt(123),"{'agent_type': 'openai-functions', 'log': 'Inv...",record_hash_2ab2cd6637940f8777b4f641cc3ceb3e,text
2,session_hash_dc51233f49a100fd2351e2108507aeb6,2023-08-10 16:27:18.827158,3,system,Observation,Answer: 11.090536506409418,{'agent_type': 'openai-functions'},record_hash_2ab2cd6637940f8777b4f641cc3ceb3e,text
3,session_hash_dc51233f49a100fd2351e2108507aeb6,2023-08-10 16:27:19.916726,3,assistant,AgentFinish,The square root of 123 is approximately 11.0905.,{'agent_type': 'openai-functions'},record_hash_2ab2cd6637940f8777b4f641cc3ceb3e,text
4,session_hash_dc51233f49a100fd2351e2108507aeb6,2023-08-10 16:27:20.072039,0,user,Input,how much is Euler's number divided by PI,{'agent_type': 'openai-functions'},record_hash_be30377df14d9c933b9c939cff5cc922,text
5,session_hash_dc51233f49a100fd2351e2108507aeb6,2023-08-10 16:27:21.788375,0,assistant,Action[Calculator],e / pi,"{'agent_type': 'openai-functions', 'log': 'Inv...",record_hash_be30377df14d9c933b9c939cff5cc922,text
6,session_hash_dc51233f49a100fd2351e2108507aeb6,2023-08-10 16:27:23.398259,3,system,Observation,Answer: 0.8652559794322651,{'agent_type': 'openai-functions'},record_hash_be30377df14d9c933b9c939cff5cc922,text
7,session_hash_dc51233f49a100fd2351e2108507aeb6,2023-08-10 16:27:24.343953,3,assistant,AgentFinish,Euler's number divided by PI is approximately ...,{'agent_type': 'openai-functions'},record_hash_be30377df14d9c933b9c939cff5cc922,text


In [9]:
session_manager.get_messages()  # messages extracted from all sessions

,session_id,ts,call_idx,source,label,content,metadata_,record_id,content_type
0,session_hash_dc51233f49a100fd2351e2108507aeb6,2023-08-10 16:27:16.091494,0,user,Input,what is the square root of 123,{'agent_type': 'openai-functions'},record_hash_2ab2cd6637940f8777b4f641cc3ceb3e,text
1,session_hash_dc51233f49a100fd2351e2108507aeb6,2023-08-10 16:27:17.400077,0,assistant,Action[Calculator],sqrt(123),"{'agent_type': 'openai-functions', 'log': 'Inv...",record_hash_2ab2cd6637940f8777b4f641cc3ceb3e,text
2,session_hash_dc51233f49a100fd2351e2108507aeb6,2023-08-10 16:27:18.827158,3,system,Observation,Answer: 11.090536506409418,{'agent_type': 'openai-functions'},record_hash_2ab2cd6637940f8777b4f641cc3ceb3e,text
3,session_hash_dc51233f49a100fd2351e2108507aeb6,2023-08-10 16:27:19.916726,3,assistant,AgentFinish,The square root of 123 is approximately 11.0905.,{'agent_type': 'openai-functions'},record_hash_2ab2cd6637940f8777b4f641cc3ceb3e,text
4,session_hash_dc51233f49a100fd2351e2108507aeb6,2023-08-10 16:27:20.072039,0,user,Input,how much is Euler's number divided by PI,{'agent_type': 'openai-functions'},record_hash_be30377df14d9c933b9c939cff5cc922,text
5,session_hash_dc51233f49a100fd2351e2108507aeb6,2023-08-10 16:27:21.788375,0,assistant,Action[Calculator],e / pi,"{'agent_type': 'openai-functions', 'log': 'Inv...",record_hash_be30377df14d9c933b9c939cff5cc922,text
6,session_hash_dc51233f49a100fd2351e2108507aeb6,2023-08-10 16:27:23.398259,3,system,Observation,Answer: 0.8652559794322651,{'agent_type': 'openai-functions'},record_hash_be30377df14d9c933b9c939cff5cc922,text
7,session_hash_dc51233f49a100fd2351e2108507aeb6,2023-08-10 16:27:24.343953,3,assistant,AgentFinish,Euler's number divided by PI is approximately ...,{'agent_type': 'openai-functions'},record_hash_be30377df14d9c933b9c939cff5cc922,text
